# Virtual inertia scheduling (VIS) for IEEE39

In [13]:
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import andes
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
import torch
import os

logger = logging.getLogger(__name__)

from visopf import vis1, vis2, loadnn

## load data

In [14]:
# ----- get andes case from excel ------
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis_opf.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

In [15]:
# ----- get neural network data ------
nn_path = '/VIS_opf/NN_train'
nn, norm = loadnn(nn_path)

## build optimization model and solve

In [16]:
# ----- set sim scenario -----
ss = vis2(norm=norm, nn=nn, dpe=0.05, rocof_lim=0.033, nadir_lim=0.01)

# ----- set vsg gen -----
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']
ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

# ------ set gen cost ------
# iloc[0] - iloc[9]
# iloc[9]: slack bus
# iloc[0],[5],[8],[9]: vsg
ss.cost['c1'].iloc[0] = 1
ss.cost['c1'].iloc[5] = 0.1
ss.cost['c1'].iloc[9] = 1.5
ss.update_dict()

# ss.costdict

Note: Control (dynamic) parameters are renormalized based on case Sbase rather then to andes base


In [17]:
ss.build()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [18]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1040 rows, 678 columns and 11758 nonzeros
Model fingerprint: 0x34a0a1eb
Variable types: 358 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [1e-07, 1e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e-01, 1e+01]
  RHS range        [5e-03, 2e+03]
Found heuristic solution: objective 56.3751019
Presolve removed 753 rows and 450 columns
Presolve time: 0.01s
Presolved: 287 rows, 228 columns, 3784 nonzeros
Variable types: 163 continuous, 65 integer (65 binary)

Root relaxation: objective 5.286856e+01, 179 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

Msys and Dsys are normlized by devise Sbase, transform to andes Sbase when do TDS


--------------------- Results -------------------
Total Cost: 52.872
RoCof prediction: 0.00717323 ; RoCof limit: 0.033
Nadir prediction: -0.00124301 ; Nadir limit 0.01


(        gen      Sn        pg       pru  prd
 0      PV_1  10.400  4.505841  0.032994  0.0
 1      PV_2   8.360  7.000000  0.000000  0.0
 2      PV_3   8.437  8.000000  0.000000  0.0
 3      PV_4  11.748  7.000000  0.000000  0.0
 4      PV_5  10.802  7.000000  0.000000  0.0
 5      PV_6  10.857  7.991153  0.008847  0.0
 6      PV_7  10.252  7.000000  0.000000  0.0
 7      PV_8   9.702  6.967006  0.032994  0.0
 8      PV_9  16.841  0.100000  0.032994  0.0
 9  Slack_10  11.990  3.000000  0.000000  0.0,
     gen  Mvsg  Dvsg    pg_vsg   pru_vsg  prd_vsg  pmax_vsg  pmin_vsg
 0  PV_1   6.0   3.0  4.505841  0.032994      0.0       8.0       0.1
 1  PV_6   2.0   0.5  7.991153  0.008847      0.0       8.0       0.1
 2  PV_8   6.0   3.0  6.967006  0.032994      0.0       7.0       0.1
 3  PV_9   6.0   3.0  0.100000  0.032994      0.0      14.0       0.1,
 {'Msys': 6.970359088061873,
  'Dsys': 1.6520947087915607,
  'Rsys': 20.933442043222,
  'Fsys': 12.71655652795142})